In [4]:
import cobra
import pandas as pd
import re
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from pathlib import Path
import sys
sys.path.append('../code/')
import leakage, utils
import pubchempy as pcp
import seaborn as sns

In [5]:
model = cobra.io.read_sbml_model('../models/e_coli/momentiJO1366.xml')
model.reactions.DHAPT.knock_out()
model.reactions.THRA.knock_out()
model.reactions.THRA2.knock_out()
model.reactions.F6PA_TG_forward.knock_out()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [6]:
met_info_df = pd.read_csv("../data/met_info_curated.csv", encoding = "ISO-8859-1", index_col = 0)

In [7]:
met_info_df.head()

,Metabolite name,Metabolite id,Value,Uncertainty,Mass,Charge,Phosphate,Topological Polar Surface Area [Å],Concentration
0,Glucose-6-phosphate,g6p,17.65,0.53,258.119901,-2.0,1,165.0,0.003000
1,Fructose-6-phosphate,f6p,29.29,0.60,258.119901,-2.0,1,165.0,0.003000
2,"Fructose-1,6-bisphosphate",fdp,34.21,0.72,336.083922,-4.0,2,215.0,0.015200
3,Dihydroxyacetonephosphate,dhap,50.40,1.74,168.041961,-2.0,1,104.0,0.000374
4,Glyceraldehyde-3-phosphate,g3p,15.04,0.24,168.041961,-2.0,1,104.0,NaN


# Add log P to met info

In [8]:
xlogp_dict = {}
for i, row in met_info_df.iterrows():
    met_ids = row['Metabolite id'].split(',')
    xlogp_temp_list = []
    for m_id in met_ids:
        m = model.metabolites.get_by_id('{0}_c'.format(m_id.strip(' ')))
        result = pcp.get_compounds(m.name, 'name')
        for r in result:
            if r.xlogp:
                print(i, m.id, m.formula, r.molecular_formula, r.xlogp)
                xlogp_temp_list.append(r.xlogp)
    if len(xlogp_temp_list):
        xlogp_dict[i] = np.mean(xlogp_temp_list)
    else:
        xlogp_dict[i] = np.nan

0 g6p_c C6H11O9P C6H13O9P -4.5
0 g6p_c C6H11O9P C6H13O9P -4.5
1 f6p_c C6H11O9P C6H13O9P -4.3
3 dhap_c C3H5O6P C3H7O6P -2.5
4 g3p_c C3H5O6P C3H7O6P -2.7
5 2pg_c C3H4O7P C3H7O7P -2.6
5 3pg_c C3H4O7P C3H7O7P -2.6
6 pep_c C3H2O6P C3H5O6P -1.1
7 pyr_c C3H3O3 C3H3O3- -0.6
8 6pgc_c C6H10O10P C6H13O10P -4.5
9 r5p_c C5H9O8P C5H11O8P -3.6
10 ru5p__D_c C5H9O8P C5H11O8P -3.7
10 ru5p__L_c C5H9O8P C5H11O8P -3.7
10 xu5p__D_c C5H9O8P C5H11O8P -3.7
10 xu5p__L_c C5H9O8P C5H11O8P -3.7
11 e4p_c C4H7O7P C4H9O7P -3.3
12 s7p_c C7H13O10P C7H15O10P -5
13 accoa_c C23H34N7O17P3S C23H38N7O17P3S -5.6
14 cit_c C6H5O7 C6H5O7-3 0.2
15 acon_C_c C6H3O6 C6H6O6 -1
16 icit_c C6H5O7 C6H8O7 -1.8
17 akg_c C5H4O5 C5H6O5 -0.9
18 fum_c C4H2O4 C4H2O4-2 1
18 fum_c C4H2O4 C4H4O4 -0.3
19 mal__L_c C4H4O5 C4H6O5 -1.3
19 mal__L_c C4H4O5 C4H3O5-3 1.1
19 mal__D_c C4H4O5 C4H6O5 -1.3
20 glx_c C2H1O3 C2HO3- 0.3
23 nad_c C21H26N7O14P2 C21H27N7O14P2 -6
23 nad_c C21H26N7O14P2 C21H28N7O14P2+ -5.9
23 nad_c C21H26N7O14P2 C21H26N7O14P2- -6.1
24 n

In [9]:
# Fix errors
xlogp_dict[18] = -0.041#https://mimedb.org/metabolites/MMDBc0054486
xlogp_dict[19] = -1.1 #$https://mimedb.org/metabolites/MMDBc0000773
xlogp_dict[2]  = -5.9# https://pubchem.ncbi.nlm.nih.gov/compound/Fructose-1_6-bisphosphate#section=Computed-Properties
xlogp_dict[47] = -0.8#https://pubchem.ncbi.nlm.nih.gov/compound/1492348#section=Chemical-and-Physical-Properties

In [10]:
met_info_df['log P'] = met_info_df.index.map(xlogp_dict)

In [11]:
met_info_df.to_csv("../data/met_info_curated.csv")